# Laboratorio 1

###Inteligencia de Negocios

###Integrantes:
    Juan Diego Cardona 201819447
    Nicolas Ortega XXXXX
    Camila Teran XXXX

# Análisis Exploratorio y Perfilamiento de Datos

Realizar un análisis exploratorio y perfilamiento de datos que le permita entenderlos, al igual que identificar el nivel de calidad de los datos y las tareas de transformación que se requieren para construir modelos de clasificación utilizando técnicas como KNN y árboles de decisión entre otras.
Adicionalmente, la entidad desea incluir como entregable, tableros de control para apoyar tareas de entendimiento de datos y nivel de calidad de los mismos, para lo cual sugiere mostrar estadisticas tales como cantidad de columnas, cantidad de datos, valores nulos, promedio de cada variable entre otros posibles datos que puedan proveer información útil a SaludAlpes.

### 1. Importación de librerías 

En las siguientes líneas de código se importan las librerías y herramientas necesarias para desarrollar el caso de uso.

In [ ]:
# Librerías para manejo de datos
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import numpy as np
np.random.seed(3301)
import pandas as pd
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Librerías para la visualización
import matplotlib as mplt
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns 
from sklearn import tree

### 2. Carga de los datos
A través de la librería **pandas** podemos realizar la carga de datos desde diferentes fuentes de información, en este caso se realizará la carga de un archivo plano csv (archivo separado por comas).

In [ ]:
sns.__version__
mplt.__version__

In [ ]:
# Se cargan los datos. 
df_tracks=pd.read_csv('PrepTracks.csv', sep=',', encoding = 'utf-8', index_col=0)

In [ ]:
# Cantidad de datos y número de variables
df_tracks.shape

In [ ]:
# Mostrar los datos
df_tracks.head()

In [ ]:
# Podemos ver los tipos de todas la variables.
df_tracks.dtypes

In [ ]:
# Y hacer una descripción de los datos
df_tracks.describe()

In [ ]:
fig=plt.figure(figsize=(12,8))
ax = sns.boxplot(x="popularity", data=df_tracks, orient="v")

In [ ]:
# Se observa que hay ausencias, en particular en la variable "decibel_range". Veamos cuantas, para todas las variables:
df_tracks.isnull().sum()

# Justificación de preprocesamiento y limpieza de Datos

Primero vamos a ejecutar los pasos de limpieza de los datos, relacionados el tratamiento de ausencias y registros duplicados.

In [ ]:
# Es recomendable que todos los pasos de limpieza y preparación se realicen sobre otro archivo.
df_tracks_t = df_tracks
# Primero eliminaremos aquellas variables con muchas ausencias.
df_tracks_t = df_tracks_t.drop(['decibel_range', 'second_artist'], axis=1)

In [ ]:
# Eliminación registros con ausencias
#¿Porqué realizar este paso?
df_tracks_t = df_tracks_t.dropna()
# Eliminación de registros duplicados.
df_tracks_t = df_tracks_t.drop_duplicates()

En este punto es importante reflexionar en otra alternativa para no perder registros por celdas con valores nulos. ¿Cuándo tendrá sentido utilizar una estrategia distinta a eliminar los registros?

In [ ]:
df_tracks_t.describe()

In [ ]:
# Cantidad de datos y número de variables
df_tracks_t.shape

Es muy importante analizar el objetivo del modelo analítico a construir antes de iniciar a tomar decisiones. En este caso, por ejemplo, se debe aplicar la regla dada por el negocio, sobre qué significa una canción popular y que no, para generar la nueva variable que será la clase (variable objetivo).

Esta nueva columna la derivaremos a partir de la variable "popularity" utilizando un umbral que nos permita determinar cuándo una canción es popular. Si el valor en esta variable es mayor que el umbral entonces ese registro se clasifica como "popular" y será indicado con un "1", de lo contrario será "no popular y esta categoría la representaremos con un "0".

In [ ]:
# Podemos ver los estadísticos de la variable "popularity"
df_tracks_t['popularity'].describe()

In [ ]:
#El observar las variables de forma gráfica puede ayudar al negocio a decidir el punto a partir del cual se tienen canciones populares.
fig=plt.figure(figsize=(12,8))
ax = sns.boxplot("popularity", data=df_tracks_t, orient="v")

In [ ]:
# Ahora definimos la función que nos va a permitir construir nuestra clase.
def label_popularity (row):
    if row['popularity'] > 33 :
        return 1
    return 0
df_tracks_t['popularity_label']=df_tracks_t.apply (lambda row: label_popularity(row), axis=1)

In [ ]:
df_tracks_t.head()

In [ ]:
# Cantidad de datos de cada clase
pd.value_counts(df_tracks_t['popularity_label'])

In [ ]:
ax = sns.countplot(x='popularity_label', data=df_tracks_t)

In [ ]:
# Eliminaremos las variables que consideramos no son útiles para la tarea que queremos resolver, como los identificadores.
df_tracks_t = df_tracks_t.drop(['id', 'id_artists'], axis=1)

Un aspecto muy importante para tener en cuenta son los requerimientos de entrada de los algoritmos de aprendizaje implementados en las librerías utilizadas. En particular, scikit-learn requiere que todos los atributos sean numéricos y que la variable objetivo, en una tarea de clasificación, esté codificada con números.

In [ ]:
# Vamos a seleccionar de nuestro conjunto solo los atributos numéricos.
number_cols = df_tracks_t.dtypes[(df_tracks_t.dtypes == np.int64) | (df_tracks_t.dtypes == np.float64)].index 
number_cols = df_tracks_t.select_dtypes(include = ['int64','float']).columns
number_cols

In [ ]:
df_tracks_m = df_tracks_t[number_cols]

In [ ]:
# Podemos ver como quedaron los datos
df_tracks_m.head()

In [ ]:
# Revisión del total de registros
df_tracks_m.shape

# Clasificadores:

## K-Nearest-neighbours

En este punto, deben justificar las decisiones más importantes asociadas al proceso, tales como los criterios utilizados para la selección de hiperparámetros y las modificaciones a los datos para construir cada clasificador.

## Arboles de Decisión

En este punto, deben justificar las decisiones más importantes asociadas al proceso, tales como los criterios utilizados para la selección de hiperparámetros y las modificaciones a los datos para construir cada clasificador.

## Clasificador de Libre Elección

En este punto, deben justificar las decisiones más importantes asociadas al proceso, tales como los criterios utilizados para la selección de hiperparámetros y las modificaciones a los datos para construir cada clasificador.

# Análisis de Resultados Obtenidos y Utilidad de los Modelos

# Comparación de modelos obtenidos y Recomendación